23行目変更:  
header = { 'gg':'gg\_' ,'silva':'silva\_' ,'rdp':'rdp_' }  
→header = { 'gg':'gg','silva':'silva','rdp':'rdp' }に変更し他ファイルと統一

In [10]:
import os, glob

In [11]:
#### configurations ####
is_taxonomy = 'yes'
is_denovopicking = 'yes'
is_mapping = 'yes'

In [12]:
## softwares ##
vsearch = '/Users/snakaoka/ws/apps/vsearch/bin/vsearch'
## input / output file ##
input_dir = '/Users/snakaoka/Desktop/PRJNA000000'
output_dir = '/Users/snakaoka/Desktop/PRJNA000000'
config_file_name = 'config.csv'

In [13]:
## reference file ##
gg97fasta = '/Users/snakaoka/ws/ref/gg_13_5_otus/rep_set/97_otus.fasta'
gg97tax = '/Users/snakaoka/ws/ref/gg_13_5_otus/taxonomy/97_otu_taxonomy.txt'
# Silva #
silva_fasta = '/Users/snakaoka/ws/ref/silva_128/rep_set/97_otus_16S.fasta'
silva_tax = '/Users/snakaoka/ws/ref/silva_128/taxonomy/consensus_taxonomy_all_levels.txt'
# RDP #
rdp_fasta = '/Users/snakaoka/ws/ref/trainset16_022016.rdp/trainset16_022016.rdp.fasta'
rdp_tax = '/Users/snakaoka/ws/ref/trainset16_022016.rdp/trainset16_022016.rdp.tax'

fastalist = { 'gg':gg97fasta,'silva':silva_fasta,'rdp':rdp_fasta }
taxlist = { 'gg':gg97tax,'silva':silva_tax,'rdp':rdp_tax }
header = { 'gg':'gg','silva':'silva','rdp':'rdp' }

reflist = ['gg','silva','rdp']
ref = reflist[1]

In [14]:
## parameters ##
maxdiffs = 5 #10
minmergelen = 1
maxmergelen = 600
minseqlength = 50
maxee_rate = 0.5/100 #%
minsize = 2
thr = 0.97
threadsnum = 8

## file names ##
filtered_file_name = 'filtered.fasta'
centroids_file_name = 'centroids.fasta'
denovootu_file_name = header[ref]+'denovootu.txt'
hits_file_name = header[ref]+'hits.txt'
refotuname_file_name = header[ref]+'otu_name.txt'
refotuID_file_name = header[ref]+'otu.txt'

In [15]:
## filtering ##
filtered_file = os.path.join(output_dir,filtered_file_name)
centroids_file = os.path.join(output_dir,centroids_file_name)

In [16]:
## Assign taxonomy ##
hits_file = os.path.join(output_dir,hits_file_name)
if is_taxonomy=='yes':
    cmd = '%s --usearch_global %s --db %s --strand both --id %s --threads %s --userfields \"query+target+id\" --userout %s' % (vsearch,centroids_file,fastalist[ref],str(thr),str(threadsnum),hits_file)
    os.system(cmd)
else:
    msg = 'Skip taxonomy assignment.'
    print( msg )

In [17]:
## de novo picking ##
denovootu_file = os.path.join(output_dir,denovootu_file_name)
if is_denovopicking=='yes':
    cmd = '%s --usearch_global %s --db %s --strand both --id %s --threads %s --otutabout %s' % (vsearch,filtered_file,centroids_file,str(thr),str(threadsnum),denovootu_file)
    os.system(cmd)
else:
    msg = 'Skip denovo picking.'
    print( msg )

In [18]:
## Mapping to GreenGene database ##
if is_mapping=='yes':
    refid2refname = {}
    frref = open(taxlist[ref],'r').readlines()
    for line in frref:
        line = line.replace('\n','')
        tmp = line.split('\t')
        refid2refname[tmp[0]] = tmp[1]
    ## make a mapping from "user defined sequence ID" to "greengene ID"
    myid2refid = {}
    myid2refname = {}
    frh = open(hits_file,'r').readlines()
    for line in frh:
        tmp = line.split('\t') # refID: tmp[1]
        refid = tmp[1]
        refname = refid2refname[refid]
        myids = tmp[0].split(';') # myID : tmp[0]
        myid2refid[myids[0]] = refid
        myid2refname[myids[0]] = refname

    frd = open(denovootu_file,'r').readlines()
    header = frd[0] # restore header information
    frd.pop(0) # remove the header line
    ## taxonomy name version ##
    refotu_name_file = os.path.join(output_dir,refotuname_file_name)
    fw = open(refotu_name_file,'w')
    fw.write(header)
    for line in frd:
        tmp = line.split('\t') # tmp[0]: myID
        try:
            tmp[0] = myid2refname[tmp[0]]
            fw.write('\t'.join(tmp))
        except:
            continue
    fw.close()
    ## taxonomy ID version ##
    refotu_ID_file = os.path.join(output_dir,refotuID_file_name)
    fw = open(refotu_ID_file,'w')
    fw.write(header)
    for line in frd:
        tmp = line.split('\t') # tmp[0]: myID
        try:
            tmp[0] = myid2refid[tmp[0]]
            fw.write('\t'.join(tmp))
        except:
            continue
    fw.close()
else:
    msg = 'Skip taxonomy mapping.'
    print( msg )
